# Draft Kings Fantasy Team GLPK Solver

## By Retzio Gredig 
Created 2/26/22

Last Edited 3/18/22

***

# Table of Contents<a id="Top"></a>

* [1. Project Overview](#1)<br>
    * [1.1 Description](#1.1)<br>
    * [1.2 Objectives](#1.2)<br>
    * [1.3 Constraints](#1.3)<br>
* [2. Import Packages](#2) <br>
* [3. Data](#3)<br>
* [4. Historical Model Definition](#4)<br>
    * [4.1 Define Decision Variables](#4.1)<br>
    * [4.2 Define Objective Function](#4.2)<br>
    * [4.3 Define Constraints](#4.3)<br>        
* [5. Historical Model Solution](#5)<br>
* [6. Game Model Definition](#6)<br>
    * [6.1 Data](#6.1)<br>
    * [6.2 Define Decision Variables](#6.2)<br>
    * [6.3 Define Objective Function](#6.3)<br>
    * [6.4 Define Constraints](#6.4)<br>       
* [7. Game Model Solution](#7)<br>
* [8. Final Report of Solutions Found](#8)<br>

***

## 1. Project Overview<a id=1></a>

### 1.1 Description<a id=1.1></a>
Who wants to win at Daily Fantasy Sports on Draftkings? Everybody? Anybody? Well, let’s be honest, this project isn’t going to make you win, but it is a fun optimization problem! And that’s really the point in this class. 

Imagine that we were interested in joining one of the Draftkings’ National Hockey League (NHL) contests on Feb 8, 2022. There were seven real NHL games played amongst fourteen teams. The fantasy contest involves selecting a collection of players (a lineup) in these seven games and you “play” against other people that (potentially) selected different lineups. The winner of the contest is the person whose team accumulated the most fantasy points. The fantasy points are accumulated according to a set of rules (see Figure 1) that are related to how the players perform in their real-life games. Selecting a lineup is subject to a fictitious salary cap and several additional constraints that are outlined below. The goal of this project is to select an “optimal” lineup. What is “optimal?” That is for you to decide!  


### 1.2 Objectives<a id=1.2></a>
1.	Use python to apply optimization models and solution techniques learned in class to solve a real-world problem.
2.	You will have to do some exploratory data analysis using the historical data to carefully consider your objective function coefficients. 
3.	You will solve at least two optimization problems for this project. One problem will involve selecting a lineup based on historical information. The second problem will involve selecting the optimal lineup based on how the players performed during their actual games. 
4.	Write a report summarizing your results. 

### 1.3 Constraints<a id=1.3></a>
1. The fantasy team must have nine players. 
2. The salary of your nine players may not exceed &#0036;50,000
3. You need players from at least three different teams 
4. The makeup of the team in terms of positions is
    * Two Centers (C) 
    * Three Wings (W) 
    * Two Defenders (D),
    * One Utility (C, W, or D), and 
    * One Goalie


##### [Back to Top](#Top)

## 2. Import Packages<a id=2></a>

<hr>

In [1]:
import pandas as pd
import pyomo.environ as pe
import matplotlib.pyplot as plt
import seaborn as sns

##### [Back to Top](#Top)

## 3. Data<a id=3></a>

<hr>

In [2]:
raw_DK_salaries = pd.read_csv('NHL_Data/DKSalaries.csv')
raw_skaters= pd.read_csv('NHL_Data/skaters.csv')
raw_goalies = pd.read_csv('NHL_Data/goalies.csv')
raw_skaters_actual= pd.read_csv('NHL_Data/skaters-actual.csv')                          
raw_goalies_actual= pd.read_csv('NHL_Data/goalies-actual.csv')

In [3]:
DV_Index = list(raw_DK_salaries['Name'])

In [4]:
skater_points = pd.DataFrame()
skater_points = raw_DK_salaries.loc[:,['Name', 'AvgPointsPerGame', 'Position', 'Salary', 'TeamAbbrev']]
skater_points.set_index('Name', inplace=True)

In [5]:
DV_team_index = list(raw_DK_salaries['TeamAbbrev'].unique())

In [6]:
## Get All Game Info
game_info = pd.DataFrame()
game_info['Game'] = raw_DK_salaries['Game Info'].str.split(' ').str[0] 
game_info = game_info.drop_duplicates()
game_info['T1'] = game_info['Game'].str.split('@').str[0] 
game_info['T2'] = game_info['Game'].str.rsplit('@').str[1] 

In [7]:
## Get Binary for Player Position
skater_pos = pd.DataFrame()
skater_pos.index = skater_points.index
skater_pos['C'] = (skater_points['Position'] == 'C').astype(int)
skater_pos['D'] = (skater_points['Position'] == 'D').astype(int)
skater_pos['G'] = (skater_points['Position'] == 'G').astype(int)
skater_pos['W'] = (skater_points['Position'].str[-1] == 'W').astype(int)                           

In [8]:
skater_team = pd.DataFrame()
skater_team.index = skater_points.index
for x in DV_team_index:
    skater_team[x] = ((skater_points['TeamAbbrev'] == x).astype(int) )

##### [Back to Top](#Top)

## 4. Historical Model Definition<a id=4></a>

<hr>

In [9]:
model = pe.ConcreteModel()

### 4.1 Define Decision Variables<a id=4.1></a>

In [10]:
model.skater = pe.Var(DV_Index, domain = pe.Binary)
model.teams =  pe.Var(DV_team_index, domain = pe.Binary)
model.team_set = pe.Set(initialize = DV_team_index)

### 4.2 Define Objective Function<a id=4.2></a>

In [11]:
model.obj = pe.Objective(expr = sum([skater_points.loc[c, 'AvgPointsPerGame']*model.skater[c] for c in DV_Index]),
                         sense = pe.maximize)
#model.obj.pprint()

### 4.3 Define Constraints<a id=4.3></a>

In [12]:
#Budget Constraint
model.cons_budget = pe.Constraint(expr = sum([skater_points.loc[idx, 'Salary']*model.skater[idx] 
                                             for idx in DV_Index]) <= 50000)
#Total of 9 Players Constraint
model.cons_size = pe.Constraint(expr = sum(model.skater[idx] for idx in DV_Index) == 9)

# Center Constraint
model.cons_center = pe.Constraint(expr = sum([skater_pos.loc[idx, 'C'] * model.skater[idx] 
                                             for idx in DV_Index]) >= 2)

# Wing Constraint
model.cons_wing = pe.Constraint(expr = sum([skater_pos.loc[idx, 'W'] * model.skater[idx] 
                                             for idx in DV_Index]) >= 3)

# Defence Constraint
model.cons_defence = pe.Constraint(expr = sum([skater_pos.loc[idx, 'D'] * model.skater[idx] 
                                             for idx in DV_Index]) >= 2)

# Goalie Constraint
model.cons_goalie = pe.Constraint(expr = sum([skater_pos.loc[idx, 'G'] * model.skater[idx] 
                                             for idx in DV_Index]) == 1)
    
    
#Team Constraints 3 or more teams
     
def team_rule(_, t):
    return sum([skater_team.loc[idx, t] * model.skater[idx] for idx in DV_Index]) >= model.teams[t]

def team(_):
    return sum([model.teams[i] for i in DV_team_index]) >=3

model.team_cons = pe.Constraint(model.team_set, rule = team_rule)

##### [Back to Top](#Top)

## 5. Historical Model Solution<a id=5></a>

<hr>

In [13]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model)
print(success.solver.status, success.solver.termination_condition)

ok optimal


In [14]:
obj_val = model.obj.expr()
print(f'Our team has a total of {obj_val:.2f} Draft King points!')
avg_obj_val = obj_val / 9
print(f'The players have an average of {avg_obj_val:.2f} Draft King points!')


Our team has a total of 142.70 Draft King points!
The players have an average of 15.86 Draft King points!


In [15]:
DV = [] 
for index in DV_Index:
    DV.append(model.skater[index].value)

In [16]:
solution_h = skater_points['s'] = DV
solution_h = skater_points[skater_points['s'] != 0.0]
solution_h = solution_h.drop('s', axis = 1)
solution_h = solution_h.rename(columns = {'AvgPointsPerGame':'Average DK Points Per Game', 'TeamAbbrev':'Team'})
solution_h = solution_h.sort_values('Position')
s_player_1 = pd.DataFrame()
s_player_1 = solution_h
solution_h


,Average DK Points Per Game,Position,Salary,Team
Name,,,,
Bo Horvat,9.87,C,3900,VAN
Jack Drury,14.30,C,2500,CAR
Dougie Hamilton,12.94,D,5400,NJ
Jon Lizotte,9.70,D,2500,MIN
Louis Domingue,33.50,G,7300,PIT
Alex Ovechkin,17.64,LW,8800,WAS
Max Pacioretty,17.30,LW,8200,VGK
Bryan Rust,15.68,RW,7100,PIT
Matt Boldy,11.77,RW,4300,MIN


In [17]:
t = pd.DataFrame(solution_h.sum(numeric_only = True, axis = 0))
t = t.T
t.index = ['Sum']
a = pd.DataFrame(solution_h.mean(numeric_only = True, axis = 0))
a = a.T
t.loc['Average'] = a.loc[0]
t = t.round(decimals = 2)
t

,Average DK Points Per Game,Salary
Sum,142.70,50000.00
Average,15.86,5555.56


##### [Back to Top](#Top)

## 6. Game Model Definition<a id=6></a>

<hr>

### 6.1 Data<a id=6.1></a>

In [18]:
df = raw_goalies_actual.loc[:,['player', 'w', 'ot', 'svs', 'ga', 'so']]
df['points'] = (df['w'] * 6 )  + (df['svs'] * .7) - (df['ga'] * 3.5) + (df['ot'] * 2) + (df['so'] * 4)
df.loc[df['svs'] >= 35, 'points'] = df['points']+3  
goalie_points_game = df[['player', 'points']]

In [19]:
df = raw_skaters_actual.loc[:,['player', 'g', 'a', 'p', 's', 'shp']]
df['points'] = (df['g'] * 8.5) + (df['a'] * 5) + (df['s'] * 1.5) + (df['shp'] * 2)
df.loc[df['s'] >= 5, 'points'] = df['points']+3 
df.loc[df['g'] >= 3, 'points'] = df['points']+3 
df.loc[df['p'] >= 3, 'points'] = df['points']+3
player_points_game = df[['player', 'points']]

In [20]:
all_players = pd.concat([ goalie_points_game, player_points_game])
all_players = all_players.rename(columns = {'player':'Name'})

In [21]:
df = pd.DataFrame()
df = raw_DK_salaries.loc[:,['Name', 'Position', 'Salary', 'TeamAbbrev']]
all_player_points = pd.merge(all_players, df, how = "left", on = ['Name'])
all_player_points.set_index('Name', inplace=True)

In [22]:
DV_Index = list(all_player_points.index)

In [23]:
## Get Binary for Player Position
skater_pos = pd.DataFrame()
skater_pos.index = all_player_points.index
skater_pos['C'] = (all_player_points['Position'] == 'C').astype(int)
skater_pos['D'] = (all_player_points['Position'] == 'D').astype(int)
skater_pos['G'] = (all_player_points['Position'] == 'G').astype(int)
skater_pos['W'] = (all_player_points['Position'].str[-1] == 'W').astype(int)

### 6.2 Define Decision Variables<a id=6.2></a>

In [24]:
model = pe.ConcreteModel()
model.game = pe.Var(DV_Index, domain = pe.Binary)
model.teams =  pe.Var(DV_team_index, domain = pe.Binary)
model.team_set = pe.Set(initialize = DV_team_index)

### 6.3 Define Objective Function<a id=6.3></a>

In [25]:
model.obj = pe.Objective(expr = sum([all_player_points.loc[c, 'points']*model.game[c] for c in DV_Index]),
                         sense = pe.maximize)

### 6.4 Define Constraints<a id=6.4></a>

In [26]:
#Budget Constraint
model.cons_budget = pe.Constraint(expr = sum([all_player_points.loc[idx, 'Salary']*model.game[idx] 
                                             for idx in DV_Index]) <= 50000)
#Total of 9 Players Constraint
model.cons_size = pe.Constraint(expr = sum(model.game[idx] for idx in DV_Index) == 9)

# Center Constraint
model.cons_center = pe.Constraint(expr = sum([skater_pos.loc[idx, 'C'] * model.game[idx] 
                                             for idx in DV_Index]) >= 2)

# Wing Constraint
model.cons_wing = pe.Constraint(expr = sum([skater_pos.loc[idx, 'W'] * model.game[idx] 
                                             for idx in DV_Index]) >= 3)

# Defence Constraint
model.cons_defence = pe.Constraint(expr = sum([skater_pos.loc[idx, 'D'] * model.game[idx] 
                                             for idx in DV_Index]) >= 2)

# Goalie Constraint
model.cons_goalie = pe.Constraint(expr = sum([skater_pos.loc[idx, 'G'] * model.game[idx] 
                                             for idx in DV_Index]) == 1)

# Team Constraints 3 or more teams
     
def team_rule(_, t):
    return sum([skater_team.loc[idx, t] * model.game[idx] for idx in DV_Index]) >= model.teams[t]

def team(_):
    return sum([model.teams[i] for i in DV_team_index]) >=3

model.team_cons = pe.Constraint(model.team_set, rule = team_rule)
                             

##### [Back to Top](#Top)

## 7. Game Model Solution<a id=7></a>

<hr>

In [27]:
opt = pe.SolverFactory('glpk')
success = opt.solve(model)
print(success.solver.status, success.solver.termination_condition)

ok optimal


In [28]:
obj_val = model.obj.expr()
print(f'Our team has a total of {obj_val:.2f} Draft King points!')
avg_obj_val = obj_val / 9
print(f'The players have an average of {avg_obj_val:.2f} Draft King points!')

Our team has a total of 223.60 Draft King points!
The players have an average of 24.84 Draft King points!


In [29]:
DV = [] 
for index in DV_Index:
    DV.append(model.game[index].value)

In [30]:
all_player_points['s'] = DV
solution_h = all_player_points[all_player_points['s'] != 0.0]
solution_h = solution_h.drop('s', axis = 1)
solution_h = solution_h.rename(columns = {'AvgPointsPerGame':'Average DK Points Per Game', 'TeamAbbrev':'Team'})
solution_h = solution_h.sort_values('Position')
solution_h

,points,Position,Salary,Team
Name,,,,
Michael McLeod,20.0,C,2500,NJ
Boone Jenner,19.5,C,5500,CLS
Oliver Ekman-Larsson,21.0,D,3100,VAN
Ryan Graves,20.5,D,4000,NJ
Tristan Jarry,32.1,G,7500,PIT
Danton Heinen,21.5,LW,2500,PIT
Brady Tkachuk,30.5,LW,6000,OTT
Patrik Laine,35.5,RW,4800,CLS
David Pastrnak,23.0,RW,8500,BOS


In [31]:
t = pd.DataFrame(solution_h.sum(numeric_only = True, axis = 0))
t = t.T
t.index = ['Sum']
a = pd.DataFrame(solution_h.mean(numeric_only = True, axis = 0))
a = a.T
t.loc['Average'] = a.loc[0]
t = t.round(decimals = 2)
t

,points,Salary
Sum,223.60,44400.00
Average,24.84,4933.33


##### [Back to Top](#Top)

## 8. Final Report of Solutions Found<a id=8></a>

<hr>

In [32]:
df = pd.merge(s_player_1, all_players, how = "left", on = ['Name'])
df =  df[['Name', 'points']]
df

,Name,points
0,Bo Horvat,16.5
1,Jack Drury,NaN
2,Dougie Hamilton,NaN
3,Jon Lizotte,NaN
4,Louis Domingue,NaN
5,Alex Ovechkin,8.0
6,Max Pacioretty,1.5
7,Bryan Rust,19.5
8,Matt Boldy,3.0


The sum of my DraftKings points for the model based on previous games was 142.7 points. The sum of Draft King points based on the actual game was 223.6. This 80.9 point difference was larger than expected. We also ended up not having any of the players from our historical model in the actual version. This is partialy because we were basing our team on a smaller amount of data than needed to accurately predict a game with such a high amount of variables like hockey. Also, there will be more outliers in a single game compared to the averages for the season. This is also because four of our players that we picked ( Jack Drury, Dougie Hamilton, Jon Lizotte, and Louis Dominque) did not play in the games that night. We would have had a total of 48.5 Draft King Points with the team we picked. The best two we picked in the original model were Bryan Rust with 19.5 Draft King points and Bo Horvat with 16.5 points. However, both of those players had fewer draft king points than the average points of our team from the model of the actual night. 

##### [Back to Top](#Top)